(chpt:theory)=
# Theory

% Add subsection ToC, see https://jupyterbook.org/en/stable/structure/configure.html#add-a-table-of-contents-to-a-pages-content
```{tableofcontents}
```


% STUB

% TODO: plotly HV wrappers, see https://github.com/executablebooks/jupyter-book/issues/1815 - NOW fixed.

% UPDATE: now in place, working in Render-debug notebook, but not here - even after clean build? Weird.

% TODO: Fix plots (HV & plotly) in PDF output, currently missing here and Render-debug notebook.

% 17/11/22 Rendering now OK for Plotly, ongoing notes: https://github.com/phockett/Quantum-Metrology-with-Photoelectrons-Vol3/issues/2

% 22/11/22 Rendering now OK for Plotly in HTML and PDF, although need to execute notebooks for both cases to set exporters correctly, see issue as above for more.